# Anomaly Detection

This notebook is based on the Exercise 8 of the Manchine Learning Course in Coursera by A. Nguyen. (https://es.coursera.org/learn/machine-learning#)


This notebook is in github at https://github.com/jjmurillo/macler/blob/main/Exercises/AnomalyDetectionExercise.ipynb

You may open this notbook at 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jjmurillo/macler/blob/main/Exercises/AnomalyDetectionExercise.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


Also, you can try Binder, there go into the exercises folder. [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jjmurillo/macler/main)


## Introduction

In anomaly detection a solution to the problem is to estimate the probability density function to later estimate a threshold below which a sample is considered as anomalous.

In this exercise we ask first for an estimation of the probability density function assuming a Gaussian distribution. Then, for a training set, we compute a threshold.

We start with a low dimensional problem to end with anomaly detection for a larger one.

## Probability density function

We aim at detection when a server is behaving not properly. We have a pair for data: the throughput in Mbps and the latency or delay in answering to a request. We have some test samples, one for each computer server, and we conjecture that a few of them are not working properly while the rest do. 

Load your data,

In [2]:
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sb
from scipy.io import loadmat
%matplotlib inline

In [3]:
data = loadmat('ex8data1.mat')
X = data['X'] #Test data
X.shape


(307, 2)

In [4]:
yval = data['yval'] #Data to validate, to compute the threshold. yval is 1 if it was an anomaly, 0 otherwise
Xval = data['Xval']
Xval.shape

(307, 2)

## Anomaly detection

Our first task is to use a Gaussian model to detect if an unlabeled example from a data set should be considered anomalus.  We have a simple 2-dimensional data set to start off with to help visualize what the algorithm is doing.  Let's pull in and plot the data.

We ask: Use the data in $X$ to model the probability density function (pdf). In particular,

1.- Write a function to estimate the pdf of a Gaussian multidimensional and estimate the values of the probability density function (pdf), recall that the multidimensional Gaussian is given by

\begin{equation}
 \mathbf{X}\sim {N}({\mathbf{m}_{\mathbf{X}}},{\mathbf{C}_{\mathbf{X}}}) 
 =\frac{1}{(2\pi)^{d/2}|\mathbf{C}_{\mathbf{X}}|^{1/2}} \mathrm{e}^{-\frac{1}{2}(\mathrm{x}-\mathbf{m}_{\mathbf{X}})^\top\mathbf{C} _{\mathbf{X}}^{-1}(\mathrm{x}-\mathbf{m}_{\mathbf{X}})}
 \end{equation}

Solution: the function can be as follows,

In [ ]:
def multivariateGaussian(x, mu, cov):
    '''
    Caculate the multivariate normal density (pdf)
      of dimension n and 
    Keyword arguments:
        X = numpy array of a "d x n" or "n x d" sample vector
        mu = numpy array of a "d x 1" or tuple of d entries with mean vector
        cov = "numpy array of a d x d" covariance matrix
    '''
    
    '''WRITE HERE YOUR FUNCTION'''

We check for the function

In [ ]:
#Test with 1 dimension and 2 samples
x = np.array([[0],[0]])
mu  = np.array([[0]])
cov = np.eye(1) 
print(multivariateGaussian(x, mu, cov))
    # prints 0.3989 0.3989

#Test with 2 dimension and 1 sample
x = np.array([[0,0]])
mu  = np.array([[0],[0]])
cov = np.eye(2) 
    # prints 0.1591

print(multivariateGaussian(x, mu, cov))
    # prints 0.15915494309189535
#Test with 2 dimension and 307 samples
mu = X.mean(axis=0)
sigma = X.var(axis=0)
print(multivariateGaussian(X[1:15], mu, np.diag(sigma)))

We now fit a bidimensional Gaussian pdf to our data. First, we estimate the mean and variances.

In [ ]:
mu = #Your code here ...
Cov = #Your code here ...

Then estimate the values of the probability density function (pdf)

In [ ]:
#pdfX=multivariateGaussian(X, mu, np.diag(sigma)) #Using out function above
pdfX=multivariateGaussian(X, mu, Cov)

# You may want to compare to the one given in any library. We use SciPy:
from scipy import stats; 
pdfX2=stats.multivariate_normal.pdf(X, mu, Cov) #Using SciPy

We print some values,

In [ ]:
[[pdfX[1:10]],[pdfX2[1:10]]]

3.- Observe the pdf using contour. The following function might help you.


In [ ]:
def scattering(X,markerType='o',markerColor='b',axisHandle=None):
    if axisHandle is None:
        fig, axisHandle = plt.subplots(figsize=(8,8))
    axisHandle.scatter(X[:,0], X[:,1],marker=markerType,color=markerColor)
    axisHandle.axis('equal')
    return axisHandle

Solution: we only need to call the function properly,

In [ ]:
scattering(X)

Note that there area few samples far away from the main cluster, that will exhibit a low probability.

## Anomaly Detection

### Anomaly detection in a 2-dimensional space

For the last results above, and given a threshold of, e.g., $10^{-4}$, detect the anomalies.

Solution: 

We will model the data above to estimate as anomalies those samples far away from the main cluster. 

In [ ]:
#Your code here, detect the anomalies and provide their indexes

### Threshold estimation

Compute the threshold, $\epsilon$, to get a good value of 
$$
F_1=2PR/(P+R)
$$ 
where $P$ is the precission and $R$ is the \textit{recall}. The following function will help you as starting point. You will have to complete it. Check that you get a value of $\epsilon$ around $8.99e-05$.

Solution: We first complete the function 

In [ ]:
def selectThreshold(yval, pval):
    '''
    SELECTTHRESHOLD Find the best threshold (epsilon) to use for selecting
    outliers
    [bestEpsilon, bestF1] = SELECTTHRESHOLD(yval, pval) finds the best
    threshold to use for selecting outliers based on the results from a
    validation set (pval) and the ground truth (yval).
   
    '''
    best_epsilon = 0
    best_f1 = 0
    f1 = 0
    
    stepsize = (pval.max() - pval.min()) / 1000
    
    for epsilon in np.arange(pval.min(), pval.max(), stepsize):
        preds = pval < epsilon
    '''
    % ====================== YOUR CODE HERE ======================
    % Instructions: Compute the F1 score of choosing epsilon as the
    %               threshold and place the value in F1. The code at the
    %               end of the loop will compare the F1 score for this
    %               choice of epsilon and set it to be the best epsilon if
    %               it is better than the current choice of epsilon.
    %
    % Note: You can use predictions = (pval < epsilon) to get a binary vector
    %       of 0's and 1's of the outlier predictions

    % yval says it's an anomaly and so algorithm does.
    %tp = ...

    % yval says it's not an anomaly,  but algorithm says anomaly.
    %fp = ...

    % yval says it's an anomaly,  but algorithm says not anomaly.
    %fn = ...

    % precision and recall
    %prec = ...
    %rec = ...

    % F1 value;
    F1 = (2*prec*rec)/(prec+rec);

    % =============================================================
    '''
        
        if f1 > best_f1:
            best_f1 = f1
            best_epsilon = epsilon
    
    return best_epsilon, best_f1

Then we estimate the threshold using the validation data 

In [ ]:
# Estimate probabilities for validation data, then the threshold
pval= #Your code here
epsilon, f1 = select_threshold(pval, yval)
epsilon, f1

Finally, we can apply the threshold to the data set and visualize the results.

In [ ]:
# indexes of the values considered to be outliers
outliers = #Your code here


In [ ]:
ax=scattering(X,'o','b')
ax=scattering(X[outliers[0],:],'o','r',ax) #X[outliers[0],0], X[outliers[0],1]

The points in red are the ones that were flagged as outliers.  These seem pretty reasonable.  The top right point that has some separation (but was not flagged) may be an outlier too, but it's fairly close.  There's another example in the text of applying this to a higher-dimensional data set, but there's nothing new to do or look at so we'll move on to the last section.

### Anomaly detection in a larger dimensional space

Finally, a similar problem with more features, dimensions, is faced. Load the data in ex8data2.mat, with 11 dimensions, compute the threshold and detect anomalies. 

Solution:

In [ ]:
data = loadmat('ex8data2.mat')
X11 = data['X']
X11val = data['Xval']
y11val = data['yval']

Estimate parameters of the Gaussian and the values of the pdf

In [ ]:
mu11 = #Your code here
Cov11= #Your code here
p11=multivariateGaussian(X11, mu11, Cov11) 
p11val=multivariateGaussian(X11val, mu11, Cov11) 


Estimate the threshold 

In [ ]:
epsilon11, f1 = select_threshold(p11val, y11val)
epsilon11, f1

Finally, we can apply the threshold to the data set,

In [ ]:
# indexes of the values considered to be outliers
# Your code here